In [20]:
#1. Establish a connection between Python and the Sakila database.

import mysql.connector
from getpass import getpass
import pandas as pd
import pymysql
from sqlalchemy import create_engine

username = "root"
password = getpass()
host = "localhost"
port = 3306  # Change to your MySQL port if necessary
database_name = "sakila"                            

In [31]:
cnx = mysql.connector.connect(user = "root",
                             password = getpass(),
                             host = "localhost",
                             database = "sakila")

In [32]:
cnx.is_connected()

True

In [ ]:
cursor = cnx.cursor()

2. Write a Python function called `rentals_month` that retrieves rental data for a given month and year (passed as parameters) from the Sakila database as a Pandas DataFrame.

In [ ]:
engine = create_engine(f'mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database_name}')

In [37]:
def rentals_month(engine, month, year):
    
    query = f""" 
    SELECT rental_id, rental_date, return_date, inventory_id, customer_id
    FROM sakila.rental
    WHERE MONTH(rental_date) = {month}
    AND YEAR(rental_date) = {year};
    """
    rental_df = pd.read_sql_query(query, engine)
    rental_df['rental_date'] = pd.to_datetime(rental_df['rental_date'])
    return pd.DataFrame(rental_df)

In [38]:
rental_df = rentals_month(engine, 5, 2005)

rental_df

,rental_id,rental_date,return_date,inventory_id,customer_id
0,1,2005-05-24 22:53:30,2005-05-26 22:04:30,367,130
1,2,2005-05-24 22:54:33,2005-05-28 19:40:33,1525,459
2,3,2005-05-24 23:03:39,2005-06-01 22:12:39,1711,408
3,4,2005-05-24 23:04:41,2005-06-03 01:43:41,2452,333
4,5,2005-05-24 23:05:21,2005-06-02 04:33:21,2079,222
...,...,...,...,...,...
1151,1153,2005-05-31 21:36:44,2005-06-10 01:26:44,2725,506
1152,1154,2005-05-31 21:42:09,2005-06-08 16:40:09,2732,59
1153,1155,2005-05-31 22:17:11,2005-06-04 20:27:11,2048,251
1154,1156,2005-05-31 22:37:34,2005-06-01 23:02:34,460,106


In [39]:
def rental_count_month(df, month, year):
    filtered_df = df[(df['rental_date'].dt.month == month) & (df['rental_date'].dt.year == year)]
    rental_count = filtered_df.groupby('customer_id').size().reset_index(name=f'rentals_{month:02d}_{year}')
    return rental_count

In [41]:
rental_df = rentals_month(engine, 5, 2005)
rental_count_df = rental_count_month(rental_df, 5, 2005)
print(rental_count_df)

     customer_id  rentals_05_2005
0              1                2
1              2                1
2              3                2
3              5                3
4              6                3
..           ...              ...
515          594                4
516          595                1
517          596                6
518          597                2
519          599                1

[520 rows x 2 columns]
